In [1]:
import adlfs
import fsspec
import xarray as xr

local = True

if local:
    from dask.distributed import Client
    client = Client()  # set up local cluster on your laptop
    client
else:
    import dask_gateway
    cluster = dask_gateway.GatewayCluster()
    client = cluster.get_client()
    cluster.scale(5)
    cluster.adapt(minimum=5, maximum=12)
    cluster

In [2]:
from dask.distributed import PipInstall
plugin = PipInstall(packages=["xmovie"])
client.register_worker_plugin(plugin)


{'tcp://127.0.0.1:34763': {'status': 'OK'},
 'tcp://127.0.0.1:44983': {'status': 'OK'},
 'tcp://127.0.0.1:45747': {'status': 'OK'},
 'tcp://127.0.0.1:46721': {'status': 'OK'}}

In [3]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install xmovie

In [7]:
store = fsspec.get_mapper('az://daymet-zarr/daily/na.zarr', account_name="daymeteuwest")
ds = xr.open_zarr(store, consolidated=True)
ds

<xarray.Dataset>
Dimensions:                  (nv: 2, time: 14965, x: 284, y: 584)
Coordinates:
    lat                      (y, x) float32 dask.array<chunksize=(584, 284), meta=np.ndarray>
    lon                      (y, x) float32 dask.array<chunksize=(584, 284), meta=np.ndarray>
  * time                     (time) datetime64[ns] 1980-01-01T12:00:00 ... 20...
  * x                        (x) float32 -5.802e+06 -5.801e+06 ... -5.519e+06
  * y                        (y) float32 -3.9e+04 -4e+04 ... -6.21e+05 -6.22e+05
Dimensions without coordinates: nv
Data variables:
    dayl                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    lambert_conformal_conic  int16 ...
    prcp                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    srad                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    swe                      (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    time_bnds                (time, nv) datetime64[ns] dask.array<chunksize=(365, 2), meta=np.ndarray>
    tmax                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    tmin                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    vp                       (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    yearday                  (time) int16 dask.array<chunksize=(365,), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.6
    Version_data:      Daymet Data Version 4.0
    Version_software:  Daymet Software Version 4.0
    citation:          Please see http://daymet.ornl.gov/ for current Daymet ...
    references:        Please see http://daymet.ornl.gov/ for current informa...
    source:            Daymet Software Version 4.0
    start_year:        1980

In [5]:
# ds = ds.where(ds.prcp != -9999.) 
# x = ds.prcp.isel(time=0).sel(y=slice(1_000_000, -1_700_000), x=slice(-3_000_000, 3_000_000))

In [6]:
# x.plot()

In [8]:
factor = 2
da = ds.tmax.sel(time=slice("2015-01-01", "2020-12-31")).coarsen(dim={'x': factor, 'y': factor}, boundary="trim").mean().persist()
vmin, vmax = da.min(), da.max()
print(vmin.values, vmax.values)

da

# factor = 2
# # .sel(time=slice("2019-01-01", "2019-01-31")

# # fix missing nodata value
# #da = ds.where(ds.prcp != -9999.) 
# #da = da.prcp[14785:].sel(y=slice(1_000_000, -1_700_000), x=slice(-3_000_000, 3_000_000)).coarsen(dim={'x': factor, 'y': factor}, boundary="trim").mean().persist()
# #da = ds.prcp.sel(time=slice("2019-01-01", "2019-01-31"), y=slice(1_000_000, -1_700_000), x=slice(-3_000_000, 3_000_000)).coarsen(dim={'x': factor, 'y': factor}, boundary="trim").mean().persist()
# da = ds.prcp.sel(time=slice("2019-01-01", "2019-01-31")).coarsen(dim={'x': factor, 'y': factor}, boundary="trim").mean().persist()

# vmin, vmax = da.min(), da.max()
# print(vmin.values, vmax.values)


0.0 93.275


<xarray.DataArray 'prcp' (time: 31, y: 292, x: 142)>
dask.array<mean_agg-aggregate, shape=(31, 292, 142), dtype=float32, chunksize=(31, 292, 142), chunktype=numpy.ndarray>
Coordinates:
    lat      (y, x) float32 dask.array<chunksize=(292, 142), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(292, 142), meta=np.ndarray>
  * time     (time) datetime64[ns] 2019-01-01T12:00:00 ... 2019-01-31T12:00:00
  * x        (x) float32 -5.802e+06 -5.8e+06 -5.798e+06 ... -5.522e+06 -5.52e+06
  * y        (y) float32 -3.95e+04 -4.15e+04 -4.35e+04 ... -6.195e+05 -6.215e+05
Attributes:
    cell_methods:  area: mean time: sum
    grid_mapping:  lambert_conformal_conic
    long_name:     daily total precipitation
    units:         mm/day

In [9]:
import cartopy
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cmocean

import cartopy.crs as ccrs
import pandas as pd 


lcc = ds.variables["lambert_conformal_conic"].attrs

prj_kwargs = dict(
    central_latitude=lcc['latitude_of_projection_origin'],
    central_longitude=lcc['longitude_of_central_meridian'],
    standard_parallels=(lcc["standard_parallel"])
)

prj = ccrs.LambertConformal(**prj_kwargs)


def custom_plotfunc(ds, fig, tt, framedim="time", **kwargs):
    ax = plt.axes(projection=prj, frameon=False)
    #ax.set_extent([-145, -58, 9, 84])
    #ax.set_extent([-2998750, 2999250, -1698500, 999500 ], crs=prj)
    
    ds.isel({framedim:tt}).plot(ax=ax, transform=prj,
                                vmin=0, vmax=75,  
                                cmap=cmocean.cm.rain, 
                                cbar_kwargs={"aspect": 50, "shrink": 0.8},
                                robust=False
                               )
    
    date = ds.isel({"time":tt}).time.values
    ts = pd.to_datetime(str(date))  
    ax.set_title(ts.strftime('%Y %b'))

    countries = cartopy.feature.NaturalEarthFeature(category='cultural', 
        name='admin_0_countries', scale="10m", facecolor="none")
    lakes = cartopy.feature.NaturalEarthFeature('physical', 'lakes', 
        scale="110m", facecolor=cfeature.COLORS['water'])
    states = cartopy.feature.NaturalEarthFeature(category='cultural', 
        name='admin_1_states_provinces_lines', scale="10m", facecolor="none")
    
    ax.add_feature(countries, zorder=2, linewidth=0.2, edgecolor="lightgray")
    ax.add_feature(states, zorder=3, linewidth=0.2, edgecolor="lightgray")
    ax.add_feature(lakes, zorder=4, linewidth=0.3, edgecolor="lightgray")
    ax.add_feature(cartopy.feature.OCEAN, zorder=0)


In [10]:
%%time
#import warnings; warnings.simplefilter("ignore")

from xmovie import Movie

mov = Movie(da.chunk({'time':1}), custom_plotfunc, vmin=vmin.values, vmax=vmax.values, dpi=150)
mov.preview(0)
#mov.preview(1)

/srv/conda/envs/notebook/lib/python3.8/site-packages/xmovie/core.py:323: UserWarning: The provided `plotfunc` does not provide the expected number of output arguments.            Expected a function `ax,pp =plotfunc(...)` but got %i output arguments. Inserting dummy values. This should not affect output. 
  warnings.warn(


CPU times: user 381 ms, sys: 17.1 ms, total: 398 ms
Wall time: 401 ms


In [13]:
%%time
mov.save('movie.mp4', 
         parallel=local, 
         progress=True,
         verbose= not local,
         overwrite_existing=True,
         framerate=3
        )

Movie created at movie.mp4


In [14]:
from IPython.display import Video

Video("./movie.mp4")

In [11]:
client

Client Scheduler: tcp://127.0.0.1:36373 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 32.00 GiB
